In [ ]:
import replicate

my_token = "Your API Key"
client = replicate.Client(api_token = my_token)

# The meta/llama-2-70b-chat model can stream output as it's running.
for event in client.stream(
    "meta/llama-2-70b-chat",
    input={
        "top_k": 0,
        "top_p": 1,
        "prompt": "Can you write a poem about open source machine learning? Let's make it in the style of E. E. Cummings.",
        "max_tokens": 512,
        "temperature": 0.5,
        "system_prompt": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.",
        "length_penalty": 1,
        "max_new_tokens": 500,
        "min_new_tokens": -1,
        "prompt_template": "<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{prompt} [/INST]",
        "presence_penalty": 0,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

 Sure! Here's a poem about open source machine learning in the style of E. E. Cummings:

open source machine learning
a gift to the world, a treasure trove
of knowledge, freely shared
a collaborative effort, a symphony

of code and data, a work of art
a journey to the future, a new start
a chance to learn, to grow, to explore
a community of minds, a diverse score

the algorithms, they dance and play
a complex melody, a beautiful array
of patterns, of insights, of predictions
a world of possibilities, a new dimension

the models, they learn and adapt
a living, breathing, ever-changing fact
a tool for innovation, a key to unlock
a door to the future, a new path to follow

the code, it's open and free
a gift to the world, a breeze
to use, to modify, to share
a movement, a revolution, a new era

join the community, be a part
of something bigger, of a new start
together we'll shape the future, we'll make
a better world, a brighter day to break.

In [ ]:
import os
import replicate
import pandas as pd
from datasets import load_dataset

In [ ]:
# --- 1. SETUP ---
# The script will automatically use the REPLICATE_API_TOKEN environment variable.
# Ensure it is set before running.
# You can check if the token is available with:
# print(os.getenv("REPLICATE_API_TOKEN"))

# --- 2. LOAD DATASET ---
# Load the 'generation' subset of the TruthfulQA dataset
print("Loading TruthfulQA dataset...")
dataset = load_dataset("truthful_qa", "generation")
# We'll use the validation split for this example
questions = dataset['validation']['question']
print(f"Dataset loaded. Found {len(questions)} questions.")

# --- 3. DEFINE PARAMETERS ---
# Define the temperatures to test
temperatures = [0.1, 0.4, 0.7, 1.0, 1.3]

# Llama 2 70B Chat model on Replicate
LLAMA_2_70B_CHAT = "meta/llama-2-70b-chat"

# List to store all the results
results_list = []

# --- 4. MAIN PROCESSING LOOP ---
print("Starting to process questions with different temperatures...")

# You can limit the number of questions for a quick test, e.g., questions[:10]
for i, question in enumerate(questions):
    for temp in temperatures:
        try:
            print(f"Processing Q{i+1}/{len(questions)}: '{question[:50]}...' with temp={temp}")

            # The system prompt guides the model's behavior
            system_prompt = (
                "You are a helpful, respectful and honest assistant. "
                "Always answer as helpfully as possible, while being safe. "
                "Keep your answer concise, to a maximum of 5 sentences."
            )

            # The full prompt template
            prompt_template = (
                f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n"
                f"{question} [/INST]"
            )

            # Call the Replicate API
            output = client.run(
                LLAMA_2_70B_CHAT,
                input={
                    "prompt": prompt_template,
                    "temperature": temp,
                    "max_new_tokens": 150, # Limit output length
                    "min_new_tokens": -1
                }
            )

            # The output is an iterator, so we join it to get the full string
            full_response = "".join(list(output))

            # Store the result
            results_list.append({
                "Question": question,
                "Temperature": temp,
                "Answer": full_response.strip()
            })

        except Exception as e:
            print(f"An error occurred with question '{question}' at temp {temp}: {e}")
            # Optionally, store error information
            results_list.append({
                "Question": question,
                "Temperature": temp,
                "Answer": f"API_ERROR: {e}"
            })

# --- 5. SAVE RESULTS TO CSV ---
print("\nProcessing complete. Saving results to CSV file...")

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(results_list)

# Save the DataFrame to a CSV file
output_filename = 'truthfulqa_results.csv'
df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"Successfully saved results to {output_filename}")